In [35]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time

# toa dataframe
data = []
data_frame = pd.DataFrame({'name': [], 'birth': [], 'death': [], 'nationality': []})

# tạo 1 hàm để lấy thông tin về ngày tháng năm sinh
def extract_birth_or_death_date(texts):
    # tìm toàn bộ năm sinh hoặc năm mất nếu có đủ tất cả ngày tháng năm sinh
    full_date = re.findall(r'[0-9]{1,2}+\s[A-Za-z]+\s+[0-9]{4}', texts)
    if full_date:
        return full_date[0]
    # nếu chỉ có năm sinh hoặc năm mất
    year_only = re.findall(r'[0-9]{4}', texts)
    if year_only:
        return year_only[0]
    # Nếu không có ngày, tìm tháng và năm
    month_and_year = re.findall(r'[A-Za-z]+\s+[0-9]{4}', texts)
    if month_and_year:
        return month_and_year[0]
    # Nếu không tìm thấy bất kỳ phần nào, trả về chuỗi trống
    return ""
# tạo hàm để lấy năm sinh năm mất trong thẻ <li>
def extract_years(textss):
    years = re.search(r'\((\d{4})\s*[-]\s*(\d{4}))\)', textss)
    if years:
        return years.group(1), years.group(2)  # Trả về năm sinh và năm mất
    return ""  # Trả về chuỗi trống nếu không tìm thấy
# tao driver
driver = webdriver.Chrome()
# mo trang
for i in range (90,91): #66-91
    url = "https://en.wikipedia.org/wiki/List_of_painters_by_name_beginning_with_%22" + chr(i)+"%22"

    try:
         driver.get(url)

         # đợi một chút để trang tải
         time.sleep(10)

        # lấy ra tất cả ca thể ul
         ul_tags = driver.find_elements(By.TAG_NAME, "ul")
         print(len(ul_tags))
        # choọn the vi thứ 21
         ul_painters = ul_tags[20]  # list bắt đầu index bằng 0, bắt đầu lấy từ 1

         # Lấy ra tất cả th <li> thuộc ul_painters
         li_tags = ul_painters.find_elements(By.TAG_NAME, "li")
         links = []
        # Chờ tối đa 10 giây cho đến khi thẻ <a> xuất hiện
         element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "a"))
            )
         for tag in li_tags:
            try:
                # Lấy văn bản từ thẻ <li>
                text = tag.text

                # Lấy tên và quốc tịch nằm sau dấu cách và dấu phẩy, trước chữ "painter"
                if ',' in text and 'painter' in text:
                    name = text.split(',')[0].strip()  # Lấy tên trước dấu phẩy
                    nationality = re.search(r',\s*(.*?)\s*painter', text)
                    
                    if nationality:
                        nationality = nationality.group(1).strip()  # Lấy quốc tịch giữa dấu phẩy và chữ "painter"
                    else:
                        nationality = ""
                else:
                    name = text.strip()
                    nationality = ""
                if '(' in text and ')' in text:
                    years = re.search(r'\((\d{4})\s*[-]\s*(\d{4}))\)', text)
                    if years:
                        years.group(1), years.group(2)  # Trả về năm sinh và năm mất
                    else:
                        years = ""  # Trả về chuỗi trống nếu không tìm thấy
                # Lưu liên kết của họa sĩ
                link = tag.find_element(By.TAG_NAME, "a").get_attribute("href")

                # Thêm dữ liệu vào danh sách
                painter = {'name': name, 'birth': years.group(1), 'death': years.group(2), 'nationality': nationality}
                data.append(painter)

            except Exception as e:
                print(f"Lỗi khi lấy thông tin từ thẻ <li>: {e}")
                continue


    except Exception as e:
        print(f"Lỗi khi mở trang {url}: {e}")
# lay thong tin cac hoa si
for link in data:
    try:
        driver.get(link)
        time.sleep(20)

        try:
            name = driver.find_element(By.TAG_NAME, "h1").text
        except:
            name = ""
        # Lấy ngày sinh (chấp nhận chỉ năm, hoặc cả ngày/tháng/năm)
        if not painter['birth']:
            try:
                birth_element = driver.find_element(By.XPATH, "//th[text()='Born']/following-sibling::td")
                birth = extract_birth_or_death_date(birth_element.text)
                painter['birth'] = birth
            except Exception as e:
                print(f"Lỗi khi lấy ngày sinh từ {painter['link']}: {e}")
                birth = ""
        # Lấy ngày mất (chấp nhận chỉ năm, hoặc cả ngày/tháng/năm)
        if not painter['death']:
            try:
                death_element = driver.find_element(By.XPATH, "//th[text()='Died']/following-sibling::td")
                death = extract_birth_or_death_date(death_element.text)
                painter['death'] = death
            except Exception as e:
                print(f"Lỗi khi lấy ngày mất từ {painter['link']}: {e}")
                death = ""
        # Lấy quốc tịch
        try:
            nationality_element = driver.find_element(By.XPATH, "//th[text()='Nationality']/following-sibling::td")
            nationality = nationality_element.text
        except Exception as e:
            print(f"Lỗi khi lấy quốc tịch từ {link}: {e}")
            nationality = ""
        # tao dic
        painter = {'name': name, 'birth': birth, 'death': death, 'nationality': nationality}
        # chuyen thanh dataframe
        painter_df = pd.DataFrame([painter])
        # them thong tin
        data_frame = pd.concat([data_frame, painter_df], ignore_index=True)
    except:
        print("Error")
#in
print(data_frame)

#quit
driver.quit()

27
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced parenthesis at position 25
Lỗi khi lấy thông tin từ thẻ <li>: unbalanced pa

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time

# Tạo dataframe
data = []
data_frame = pd.DataFrame({'name': [], 'birth': [], 'death': [], 'nationality': []})

# Tạo 1 hàm để lấy thông tin về ngày tháng năm sinh
def extract_birth_or_death_date(texts):
    # Tìm toàn bộ ngày tháng năm sinh hoặc mất
    full_date = re.findall(r'[0-9]{1,2}+\s[A-Za-z]+\s+[0-9]{4}', texts)
    if full_date:
        return full_date[0]
    # Nếu chỉ có năm sinh hoặc mất
    year_only = re.findall(r'[0-9]{4}', texts)
    if year_only:
        return year_only[0]
    # Nếu không có ngày, tìm tháng và năm
    month_and_year = re.findall(r'[A-Za-z]+\s+[0-9]{4}', texts)
    if month_and_year:
        return month_and_year[0]
    # Nếu không tìm thấy bất kỳ phần nào, trả về chuỗi trống
    return ""
# Tạo driver
driver = webdriver.Chrome()

# Mở trang web
for i in range(90, 91):  # 66-91
    url = "https://en.wikipedia.org/wiki/List_of_painters_by_name_beginning_with_%22" + chr(i) + "%22"

    try:
        driver.get(url)

        # Đợi trang tải
        time.sleep(20)

        # Lấy ra tất cả các thẻ ul
        ul_tags = driver.find_elements(By.TAG_NAME, "ul")
        print(len(ul_tags))

        # Chọn thẻ thứ 21
        ul_painters = ul_tags[20]  # list bắt đầu index bằng 0

        # Lấy ra tất cả thẻ <li> thuộc ul_painters
        li_tags = ul_painters.find_elements(By.TAG_NAME, "li")

        for tag in li_tags:
            try:
                # Lấy văn bản từ thẻ <li>
                text = tag.text

                if ',' in text and 'painter' in text:
                    # Lấy tên trước dấu phẩy đầu tiên
                    name = text.split(',')[0].strip()
                    # Lấy năm sinh và năm mất trong dấu ngoặc, nằm trước dấu phẩy
                    birth_death = re.search(r'\((\d{4})*[-]*(\d{4})\)', text)
                    if birth_death:
                        # Tách năm sinh và năm mất
                        birth = birth_death.group(1)
                        death = birth_death.group(2)
                    else:
                        birth = death = ""  # Nếu không tìm thấy, để trống
                    # Tìm đoạn giữa dấu phẩy và chữ "painter"
                    nationality = re.search(r',\s*(.*?)\s*painter', text)
                    if nationality:
                        nationality = nationality.group(1).strip()  # Lấy quốc tịch giữa dấu phẩy và chữ "painter"
                    else:
                        nationality = ""  # Nếu không tìm thấy, để trống
                else:
                    name = text.strip()
                    birth = death = ""
                    nationality = ""
                # Lưu liên kết của họa sĩ
                link = tag.find_element(By.TAG_NAME, "a").get_attribute("href")

                # Thêm dữ liệu vào danh sách
                painter = {'name': name, 'birth': birth, 'death': death, 'nationality': nationality}
                data.extend(painter)
                
            except Exception as e:
                print(f"Lỗi khi lấy thông tin từ thẻ <li>: {e}")
                continue

    except Exception as e:
        print(f"Lỗi khi mở trang {url}: {e}")

# Lấy thông tin chi tiết của các họa sĩ từ từng link
for painter in data:
    try:
        driver.get(link)
        time.sleep(20)

        # Lấy tên (nếu không có thì giữ nguyên)
        try:
            name = driver.find_element(By.TAG_NAME, "h1").text
            painter['name'] = name
        except:
            pass

        # Lấy ngày sinh
        if not painter['birth']:
            try:
                birth_element = driver.find_element(By.XPATH, "//th[text()='Born']/following-sibling::td")
                birth = extract_birth_or_death_date(birth_element.text)
                painter['birth'] = birth
            except Exception as e:
                print(f"Lỗi khi lấy ngày sinh từ {link}: {e}")
        # lấy ngày mất
        if not painter['death']:
            try:
                death_element = driver.find_element(By.XPATH, "//th[text()='Death']/following-sibling::td")
                death = extract_birth_or_death_date(death_element.text)
                painter['death'] = death
            except Exception as e:
                print(f"Lỗi khi lấy ngày mất từ {link}: {e}")
        # Lấy quốc tịch từ trang chi tiết nếu không có trong thẻ <li>
        if not painter['nationality']:
            try:
                nationality_element = driver.find_element(By.XPATH, "//th[text()='Nationality']/following-sibling::td")
                painter['nationality'] = nationality_element.text
            except Exception as e:
                print(f"Lỗi khi lấy quốc tịch từ {link}: {e}")

        # Chuyển thông tin vào dataframe
        painter_df = pd.DataFrame([painter])
        data_frame = pd.concat([data_frame, painter_df], ignore_index=True)

    except Exception as e:
        print(f"Lỗi khi truy cập trang {link}: {e}")

# In kết quả
print(data_frame)

# Thoát driver
driver.quit()


27
Lỗi khi truy cập trang https://en.wikipedia.org/wiki/Francisco_de_Zurbar%C3%A1n: string indices must be integers, not 'str'
Lỗi khi truy cập trang https://en.wikipedia.org/wiki/Francisco_de_Zurbar%C3%A1n: string indices must be integers, not 'str'
Lỗi khi truy cập trang https://en.wikipedia.org/wiki/Francisco_de_Zurbar%C3%A1n: string indices must be integers, not 'str'
Lỗi khi truy cập trang https://en.wikipedia.org/wiki/Francisco_de_Zurbar%C3%A1n: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.70)
Stacktrace:
	GetHandleVerifier [0x00007FF64E85FDA5+29557]
	(No symbol) [0x00007FF64E7D2240]
	(No symbol) [0x00007FF64E68B6EA]
	(No symbol) [0x00007FF64E65FCD5]
	(No symbol) [0x00007FF64E70F167]
	(No symbol) [0x00007FF64E7281A1]
	(No symbol) [0x00007FF64E7072A3]
	(No symbol) [0x00007FF64E6D12DF]
	(No symbol) [0x00007FF64E6D2451]
	GetHandleVerifier [0x00007FF64EB8DCBD+3363469]
	GetHandleVerifier [0x00007FF64EBD

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

# Tạo dataframe
data = []
data_frame = pd.DataFrame({'name': [], 'birth': [], 'death': [], 'nationality': [], 'link': []})

# Tạo 1 hàm để lấy thông tin về ngày tháng năm sinh
def extract_birth_or_death_date(texts):
    # Tìm toàn bộ ngày tháng năm sinh hoặc mất
    full_date = re.findall(r'[0-9]{1,2}+\s[A-Za-z]+\s+[0-9]{4}', texts)
    if full_date:
        return full_date[0]
    # Nếu chỉ có năm sinh hoặc mất
    year_only = re.findall(r'[0-9]{4}', texts)
    if year_only:
        return year_only[0]
    # Nếu không có ngày, tìm tháng và năm
    month_and_year = re.findall(r'[A-Za-z]+\s+[0-9]{4}', texts)
    if month_and_year:
        return month_and_year[0]
    # Nếu không tìm thấy bất kỳ phần nào, trả về chuỗi trống
    return ""

# Tạo driver
driver = webdriver.Chrome()

# Mở trang web
for i in range(90, 91):  # Thay đổi khoảng từ 66-91 để duyệt các ký tự khác
    url = "https://en.wikipedia.org/wiki/List_of_painters_by_name_beginning_with_%22" + chr(i) + "%22"

    try:
        driver.get(url)

        # Đợi trang tải
        time.sleep(20)

        # Lấy ra tất cả các thẻ ul
        ul_tags = driver.find_elements(By.TAG_NAME, "ul")
        print(len(ul_tags))

        # Chọn thẻ thứ 21 (có thể thay đổi tùy thuộc vào cấu trúc trang)
        ul_painters = ul_tags[20]  # list bắt đầu index bằng 0

        # Lấy ra tất cả thẻ <li> thuộc ul_painters
        li_tags = ul_painters.find_elements(By.TAG_NAME, "li")

        for tag in li_tags:
            try:
                # Lấy văn bản từ thẻ <li>
                text = tag.text

                if ',' in text and 'painter' in text:
                    # Lấy tên trước dấu phẩy đầu tiên
                    name = text.split(',')[0].strip()
                    # Lấy năm sinh và năm mất trong dấu ngoặc, nằm trước dấu phẩy
                    birth_death = re.search(r'\((\d{4})*[-]*(\d{4})\)', text)
                    if birth_death:
                        # Tách năm sinh và năm mất
                        birth = birth_death.group(1) if birth_death.group(1) else ""
                        death = birth_death.group(2) if birth_death.group(2) else ""
                    else:
                        birth = death = ""  # Nếu không tìm thấy, để trống
                    # Tìm đoạn giữa dấu phẩy và chữ "painter"
                    nationality = re.search(r',\s*(.*?)\s*painter', text)
                    if nationality:
                        nationality = nationality.group(1).strip()  # Lấy quốc tịch giữa dấu phẩy và chữ "painter"
                    else:
                        nationality = ""  # Nếu không tìm thấy, để trống
                else:
                    name = text.strip()
                    birth = death = ""
                    nationality = ""
                
                # Lưu liên kết của họa sĩ
                link = tag.find_element(By.TAG_NAME, "a").get_attribute("href")

                # Thêm dữ liệu vào danh sách
                painter = {'name': name, 'birth': birth, 'death': death, 'nationality': nationality, 'link': link}
                data.append(painter)
                
            except Exception as e:
                print(f"Lỗi khi lấy thông tin từ thẻ <li>: {e}")
                continue

    except Exception as e:
        print(f"Lỗi khi mở trang {url}: {e}")

# Lấy thông tin chi tiết của các họa sĩ từ từng link
for painter in data:
    try:
        link = painter['link']
        driver.get(link)
        time.sleep(20)

        # Lấy tên (nếu không có thì giữ nguyên)
        try:
            name = driver.find_element(By.TAG_NAME, "h1").text
            painter['name'] = name
        except:
            pass

        # Lấy ngày sinh
        if not painter['birth']:
            try:
                birth_element = driver.find_element(By.XPATH, "//th[text()='Born']/following-sibling::td")
                birth = extract_birth_or_death_date(birth_element.text)
                painter['birth'] = birth
            except Exception as e:
                print(f"Lỗi khi lấy ngày sinh từ {link}: {e}")
                
        # Lấy ngày mất
        if not painter['death']:
            try:
                death_element = driver.find_element(By.XPATH, "//th[text()='Died']/following-sibling::td")
                death = extract_birth_or_death_date(death_element.text)
                painter['death'] = death
            except Exception as e:
                print(f"Lỗi khi lấy ngày mất từ {link}: {e}")
                
        # Lấy quốc tịch từ trang chi tiết nếu không có trong thẻ <li>
        if not painter['nationality']:
            try:
                nationality_element = driver.find_element(By.XPATH, "//th[text()='Nationality']/following-sibling::td")
                painter['nationality'] = nationality_element.text
            except Exception as e:
                print(f"Lỗi khi lấy quốc tịch từ {link}: {e}")

        # Chuyển thông tin vào dataframe
        painter_df = pd.DataFrame([painter])
        data_frame = pd.concat([data_frame, painter_df], ignore_index=True)

    except Exception as e:
        print(f"Lỗi khi truy cập trang {link}: {e}")

# In kết quả
print(data_frame)

# Thoát driver
driver.quit()


27
Lỗi khi lấy ngày sinh từ https://en.wikipedia.org/wiki/Eugeniusz_%C5%BBak: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//th[text()='Born']/following-sibling::td"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF64E85FDA5+29557]
	(No symbol) [0x00007FF64E7D2240]
	(No symbol) [0x00007FF64E68B6EA]
	(No symbol) [0x00007FF64E6DFA15]
	(No symbol) [0x00007FF64E6DFC6C]
	(No symbol) [0x00007FF64E72BB07]
	(No symbol) [0x00007FF64E70753F]
	(No symbol) [0x00007FF64E7288A3]
	(No symbol) [0x00007FF64E7072A3]
	(No symbol) [0x00007FF64E6D12DF]
	(No symbol) [0x00007FF64E6D2451]
	GetHandleVerifier [0x00007FF64EB8DCBD+3363469]
	GetHandleVerifier [0x00007FF64EBD9B47+3674391]
	GetHandleVerifier [0x00007FF64EBCEAEB+3629243]
	GetHandleVerifier [0x00007FF64E91FC66+815670]
	(No symbol) [0x00007F

In [17]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [13]:
data_frame

,name,birth,death,nationality,link
0,Ossip Zadkine,28 January 1888,25 November 1967,Russian,https://en.wikipedia.org/wiki/Ossip_Zadkine
1,Kristian Zahrtmann,31 March 1843,22 June 1917,Danish,https://en.wikipedia.org/wiki/Kristian_Zahrtmann
2,Eugeniusz Zak,,,Polish,https://en.wikipedia.org/wiki/Eugeniusz_%C5%BBak
3,Marcin Zaleski,1796,1877,Polish,https://en.wikipedia.org/wiki/Marcin_Zaleski
4,Domenichino,1581,1641,Italian,https://en.wikipedia.org/wiki/Domenico_Zampieri
5,Federico Zandomeneghi,,,Italian Impressionist,https://en.wikipedia.org/wiki/Federico_Zandome...
6,Zao Wou-Ki,1 February 1920,9 April 2013,"(1920–2013), Chinese/French",https://en.wikipedia.org/wiki/Zao_Wou-Ki
7,Harriet Zeitlin,1929,,American,https://en.wikipedia.org/wiki/Harriet_Zeitlin
8,Wolfgang Zelmer,,,German,https://en.wikipedia.org/wiki/Wolfgang_Zelmer
9,Zeng Jing,,,"1568–1650), Chinese",https://en.wikipedia.org/wiki/Zeng_Jing


In [15]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


In [16]:
# Đặt tên file
file_name = 'Painters.xlsx'

# Lưu vào file Excel
data_frame.to_excel(file_name, index=False)  # index=False để không lưu chỉ số
print('DataFrame is written to Excel file successfully.')

DataFrame is written to Excel file successfully.
